# Spreadsheet

    Make a spreadsheet using pinkfish.  This is useful for developing trading strategies.
    It can also be used as a tool for buy and sell signals that you then manually execute.

In [1]:
import datetime

import matplotlib.pyplot as plt
import pandas as pd

from talib.abstract import *

import pinkfish as pf
import pinkfish.itable as itable

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

# Show all columns.
pd.set_option('display.max_columns', None)

%matplotlib inline

In [2]:
# Set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Some global data

In [3]:
symbol = 'SPY'
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Fetch symbol data from internet; do not use local cache. 

In [4]:
ts = pf.fetch_timeseries(symbol, use_cache=False)
ts.tail()

,open,high,low,close,adj_close,volume
date,,,,,,
2024-08-15,549.50,553.36,548.88,553.07,553.07,60846800
2024-08-16,551.42,555.02,551.26,554.31,554.31,44430700
2024-08-19,554.73,559.61,553.86,559.61,559.61,39121800
2024-08-20,559.15,560.84,557.33,558.70,558.70,33732300
2024-08-21,559.77,562.11,554.73,560.62,560.62,41467000


Select timeseries between start and end.  Back adjust prices relative to adj_close for dividends and splits.

In [5]:
ts = pf.select_tradeperiod(ts, start, end, use_adj=False)
ts.head()

,open,high,low,close,adj_close,volume
date,,,,,,
1993-01-29,43.97,43.97,43.75,43.94,24.68,1003200
1993-02-01,43.97,44.25,43.97,44.25,24.86,480500
1993-02-02,44.22,44.38,44.12,44.34,24.91,201300
1993-02-03,44.41,44.84,44.38,44.81,25.18,529400
1993-02-04,44.97,45.09,44.47,45.00,25.28,531500


Add technical indicators

In [6]:
# Add 200 day MA.
ts['sma200'] = pf.SMA(ts, timeperiod=200)

# 200 sma regime filter.
ts['regime'] = pf.CROSSOVER(ts, timeperiod_fast=1, timeperiod_slow=200)

# X day sma.
ts['sma70'] = pf.SMA(ts, timeperiod=70)

# Add ATR.
ts['atr'] = ATR(ts, timeperiod=14)

# Add 7 day high, and 7 day low
ts['high7'] = pd.Series(ts.close).rolling(window=7).max()
ts['low7'] = pd.Series(ts.close).rolling(window=7).min()

# Add RSI, and 2-period cumulative RSI
ts['rsi2'] = RSI(ts, timeperiod=2)
ts['c2rsi2'] = pd.Series(ts.rsi2).rolling(window=2).sum()

# Add midpoint
ts['mp'] = (ts.high + ts.low) / 2

# Add 10 day SMA of midpoint
ts['sma10'] = pd.Series(ts.mp).rolling(window=10).mean()

# Add temporary rolling 10 day Standard Deviation of midpoint
ts['__sd__'] = pd.Series(ts.mp).rolling(window=10).std()

# Add standard deviation envelope or channel around midpoint
ts['upper'] = ts.sma10 + ts['__sd__']*2
ts['lower'] = ts.sma10 - ts['__sd__']*2

# Drop temporary columns.
ts.drop(columns=['__sd__'], inplace=True)

Finalize timeseries

In [7]:
ts, start = pf.finalize_timeseries(ts, start, dropna=True)
ts.tail()

,open,high,low,close,adj_close,volume,sma200,regime,sma70,atr,high7,low7,rsi2,c2rsi2,mp,sma10,upper,lower
date,,,,,,,,,,,,,,,,,,
2024-08-15,549.50,553.36,548.88,553.07,553.07,60846800,504.45,197.00,539.20,8.81,553.07,518.66,98.91,195.12,551.12,532.27,552.35,512.20
2024-08-16,551.42,555.02,551.26,554.31,554.31,44430700,505.14,198.00,539.73,8.45,554.31,530.65,99.08,197.98,553.14,534.31,558.35,510.26
2024-08-19,554.73,559.61,553.86,559.61,559.61,39121800,505.85,199.00,540.33,8.26,559.61,532.99,99.61,198.69,556.73,538.29,562.72,513.86
2024-08-20,559.15,560.84,557.33,558.70,558.70,33732300,506.53,200.00,540.88,7.92,559.61,533.27,83.16,182.77,559.09,541.82,567.13,516.50
2024-08-21,559.77,562.11,554.73,560.62,560.62,41467000,507.18,201.00,541.45,7.88,560.62,542.04,90.07,173.23,558.42,545.18,569.36,521.00


Select a smaller time from for use with itable

In [8]:
df = ts['2023-06-01':]
df.head()

,open,high,low,close,adj_close,volume,sma200,regime,sma70,atr,high7,low7,rsi2,c2rsi2,mp,sma10,upper,lower
date,,,,,,,,,,,,,,,,,,
2023-06-01,418.09,422.92,416.79,421.82,414.46,88865000,396.61,49.00,406.18,4.58,421.82,411.09,78.64,120.15,419.86,417.32,422.88,411.75
2023-06-02,424.50,428.74,423.95,427.92,420.46,91366700,396.60,50.00,406.60,4.74,427.92,411.09,92.76,171.40,426.35,418.23,426.20,410.27
2023-06-05,428.28,429.62,426.37,427.10,419.65,65460200,396.60,51.00,406.98,4.64,427.92,414.65,78.76,171.52,427.99,419.13,429.23,409.03
2023-06-06,426.67,428.58,425.99,428.03,420.56,64022200,396.60,52.00,407.43,4.49,428.03,417.85,84.18,162.94,427.28,419.97,431.30,408.64
2023-06-07,428.44,429.62,426.11,426.55,419.11,85373300,396.62,53.00,407.84,4.42,428.03,417.85,46.46,130.64,427.86,421.14,433.12,409.15


Use itable to format the spreadsheet.  New 5 day high has blue highlight; new 5 day low has red highlight.

In [9]:
pt = itable.PrettyTable(
    df, tstyle=itable.TableStyle(theme='theme1'), header_row=True, rpt_header=20)

pt.update_col_header_style(
    format_function=lambda x: x.upper(), text_align='right')
pt.update_row_header_style(
    format_function=lambda x: pd.to_datetime(str(x)).strftime('%Y/%m/%d'), text_align='right')

for col in range(pt.num_cols):
    if pt.df.columns[col] in ('volume', 'regime'):
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.0f'), text_align='right')
    else:
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.2f'), text_align='right')

for row in range(pt.num_rows):
    if row == 0:
        continue
    if (pt.df['high7'].iloc[row] == pt.df['close'].iloc[row]):
        col = df.columns.get_loc('high7')    
        pt.update_cell_style(rows=[row], cols=[col], color='blue')
    if (pt.df['low7'].iloc[row] == pt.df['close'].iloc[row]):
        col = df.columns.get_loc('low7')
        pt.update_cell_style(rows=[row], cols=[col], color='maroon')
    if (pt.df['regime'].iloc[row] > 0 or pt.df['close'].iloc[row] > pt.df['sma70'].iloc[row]):
        col = df.columns.get_loc('regime')
        pt.update_cell_style(rows=[row], cols=[col], color='green')

In [10]:
pt

,OPEN,HIGH,LOW,CLOSE,ADJ_CLOSE,VOLUME,SMA200,REGIME,SMA70,ATR,HIGH7,LOW7,RSI2,C2RSI2,MP,SMA10,UPPER,LOWER
2023/06/01,418.09,422.92,416.79,421.82,414.46,88865000,396.61,49,406.18,4.58,421.82,411.09,78.64,120.15,419.86,417.32,422.88,411.75
2023/06/02,424.50,428.74,423.95,427.92,420.46,91366700,396.60,50,406.60,4.74,427.92,411.09,92.76,171.40,426.35,418.23,426.20,410.27
2023/06/05,428.28,429.62,426.37,427.10,419.65,65460200,396.60,51,406.98,4.64,427.92,414.65,78.76,171.52,427.99,419.13,429.23,409.03
2023/06/06,426.67,428.58,425.99,428.03,420.56,64022200,396.60,52,407.43,4.49,428.03,417.85,84.18,162.94,427.28,419.97,431.30,408.64
2023/06/07,428.44,429.62,426.11,426.55,419.11,85373300,396.62,53,407.84,4.42,428.03,417.85,46.46,130.64,427.86,421.14,433.12,409.15
2023/06/08,426.62,429.60,425.82,429.13,421.64,61952800,396.70,54,408.31,4.38,429.13,417.85,79.10,125.57,427.71,422.77,433.19,412.36
2023/06/09,429.96,431.99,428.87,429.90,422.40,85742800,396.79,55,408.82,4.29,429.90,421.82,84.68,163.78,430.43,424.39,433.92,414.86
2023/06/12,430.92,433.88,430.17,433.80,426.23,76104300,396.89,56,409.33,4.26,433.80,426.55,95.86,180.54,432.03,425.79,435.28,416.30
2023/06/13,435.32,437.33,434.63,436.66,429.04,95899700,396.98,57,409.79,4.21,436.66,426.55,98.00,193.86,435.98,427.32,438.00,416.64
2023/06/14,437.01,439.06,433.59,437.18,429.55,100612100,397.13,58,410.26,4.30,437.18,426.55,98.32,196.32,436.32,429.18,438.86,419.50
